# **ENSTROPHY TESTING**

## **Installs**

### Firedrake

In [3]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-13 17:32:59--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-13 17:32:59 (51.7 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Gmsh

In [4]:
try:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh"
    !bash "/tmp/gmsh-install.sh"
    import gmsh  # noqa: F401
except:
    import gmsh  # noqa: F401

--2026-02-13 17:33:02--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3497 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.42K  --.-KB/s    in 0s      

2026-02-13 17:33:02 (30.1 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3497/3497]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ set +x
























################################################################################
#     This installation

## **Code**

### Mesh

In [5]:
def rect_with_hole_mesh(r=0.5, left=1.0, right=5.0, updown=1.0, h_fine=0.05, h_coarse=0.2, obstacle="circle"):
    '''
    r = hole radius
    left = space to left of hole centre
    right = space to right of hole centre
    updown = space above / below hole centre
    h = (target) mesh size
    '''
    gmsh.initialize()
    gmsh.model.add("rect_with_hole")

    # Derived rectangle bounds
    xmin = -left
    xmax = right
    ymin = -updown
    ymax = updown


    # ---------------
    #    Rectangle
    # ---------------
    p1 = gmsh.model.geo.addPoint(xmin, ymin, 0, h_coarse)
    p2 = gmsh.model.geo.addPoint(xmax, ymin, 0, h_coarse)
    p3 = gmsh.model.geo.addPoint(xmax, ymax, 0, h_coarse)
    p4 = gmsh.model.geo.addPoint(xmin, ymax, 0, h_coarse)

    l_bottom = gmsh.model.geo.addLine(p1, p2)
    l_right  = gmsh.model.geo.addLine(p2, p3)
    l_top    = gmsh.model.geo.addLine(p3, p4)
    l_left   = gmsh.model.geo.addLine(p4, p1)

    rect_loop = gmsh.model.geo.addCurveLoop(
        [l_bottom, l_right, l_top, l_left]
    )


    # ------------------------------
    #    Hole (centred at origin)
    # ------------------------------
    if obstacle=="circle":
        c  = gmsh.model.geo.addPoint(0, 0, 0, h_coarse)

        cp = gmsh.model.geo.addPoint( r, 0, 0, h_coarse)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h_coarse)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h_coarse)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h_coarse)

        a1 = gmsh.model.geo.addCircleArc(cp, c, ct)
        a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
        a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
        a4 = gmsh.model.geo.addCircleArc(cb, c, cp)
    elif obstacle=="semicircle":
        c  = gmsh.model.geo.addPoint(0, 0, 0, h_coarse)

        cp = gmsh.model.geo.addPoint(0,  0, 0, h_coarse)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h_coarse)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h_coarse)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h_coarse)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
        a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="diamond":
        cp = gmsh.model.geo.addPoint( r, 0, 0, h_coarse)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h_coarse)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h_coarse)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h_coarse)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="wedge":
        cp = gmsh.model.geo.addPoint( 0, 0, 0, h_coarse)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h_coarse)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h_coarse)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h_coarse)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="square":
        cp = gmsh.model.geo.addPoint( r,  r, 0, h_coarse)
        cn = gmsh.model.geo.addPoint(-r, -r, 0, h_coarse)
        ct = gmsh.model.geo.addPoint(-r,  r, 0, h_coarse)
        cb = gmsh.model.geo.addPoint(r,  -r, 0, h_coarse)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)

    if obstacle is not None:
        hole_loop = gmsh.model.geo.addCurveLoop([a1, a2, a3, a4])


    # -----------------------
    #    Surface with hole
    # -----------------------
    if obstacle is not None:
        surface = gmsh.model.geo.addPlaneSurface([rect_loop, hole_loop])
    else:
        surface = gmsh.model.geo.addPlaneSurface([rect_loop])

    gmsh.model.geo.synchronize()


    # ---------------------
    #    Physical groups
    # ---------------------
    gmsh.model.addPhysicalGroup(1, [l_left], tag=1)
    gmsh.model.setPhysicalName(1, 1, "inlet")

    gmsh.model.addPhysicalGroup(1, [l_right], tag=2)
    gmsh.model.setPhysicalName(1, 2, "outlet")

    gmsh.model.addPhysicalGroup(1, [l_top, l_bottom], tag=3)
    gmsh.model.setPhysicalName(1, 3, "wall")

    if obstacle is not None:
        gmsh.model.addPhysicalGroup(1, [a1, a2, a3, a4], tag=4)
        gmsh.model.setPhysicalName(1, 4, "hole")

    gmsh.model.addPhysicalGroup(2, [surface], tag=5)



    # ----------------------------
    #    Mesh refinement fields
    # ----------------------------
    if obstacle is not None:
        # Distance field
        gmsh.model.mesh.field.add("Distance", 1)
        gmsh.model.mesh.field.setNumbers(1, "CurvesList", [a1, a2, a3, a4])
        gmsh.model.mesh.field.setNumber(1, "Sampling", 100)

        # Threshold field
        gmsh.model.mesh.field.add("Threshold", 2)
        gmsh.model.mesh.field.setNumber(2, "InField", 1)
        gmsh.model.mesh.field.setNumber(2, "SizeMin", h_fine)   # Size near obstacle
        gmsh.model.mesh.field.setNumber(2, "SizeMax", h_coarse) # Size far away
        gmsh.model.mesh.field.setNumber(2, "DistMin", 1.0 * r)  # Radius where h_fine starts growing
        gmsh.model.mesh.field.setNumber(2, "DistMax", 2.0 * r)  # Distance where mesh reaches h_coarse

        # Background mesh
        gmsh.model.mesh.field.setAsBackgroundMesh(2)
    else:
        gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 1)



    # ----------
    #    Mesh
    # ----------
    gmsh.model.mesh.generate(2)
    gmsh.write("rect_with_hole.msh")
    gmsh.finalize()



    return Mesh("rect_with_hole.msh")

### Navier–Stokes

In [11]:
def navier_stokes(Re=1.0, sigma=10.0, gamma=1.0, dt=0.02, T=0.1, theta=1.0, osc_amp=None, save=True):
    # -----------
    #    Setup
    # -----------
    # Mesh
    mesh = rect_with_hole_mesh()
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    # x, y = SpatialCoordinate(mesh)

    # Normalise parameters
    Re_c = Constant(Re)
    sigma_c = Constant(sigma)
    gamma_c = Constant(gamma)
    dt_c = Constant(dt)
    if osc_amp is not None:
        osc_amp_c = Constant(osc_amp)

    # Spaces
    W = FunctionSpace(mesh, "CG", 3)
    V = FunctionSpace(mesh, "BDM", 2)
    Q = FunctionSpace(mesh, "DG", 1)
    Z = W * V * Q

    # Unknowns
    z = Function(Z)
    w, u, p = split(z)
    w_sol, u_sol, p_sol = z.subfunctions
    w_sol.rename("vorticity")
    u_sol.rename("velocity")
    p_sol.rename("pressure")

    # Tracker functions
    u_old = Function(V)

    # Tests
    c, v, q = TestFunctions(Z)

    # Solver parameters
    sp = {
        "snes_monitor" : None,
        "snes_converged_reason" : None,
        # "snes_atol" : 1e-10,
        # "snes_rtol" : 1e-10,
    }



    # -----------------------
    #    General operators
    # -----------------------
    def dev(w):
        return sym(grad(w))
    def jump(w):
        return outer(w('+') - w('-'), n('+'))
    def jump_dev(w):
        return sym(jump(w))
    def rotate(w):
        return as_vector([-w[1], w[0]])

    def h1_sym(w1, w2, u_par=0):
        h1_sym_bulk = 2 / Re_c * inner(dev(w1), dev(w2)) * dx
        h1_sym_int = 2 / Re_c * (
          - inner(avg(dev(w1)), jump_dev(w2))
          - inner(jump_dev(w1), avg(dev(w2)))
          + sigma_c / avg(h) * inner(jump_dev(w1), jump_dev(w2))
        ) * dS
        if u_par == 0:
            h1_sym_ext = (
                gamma_c * inner(w1, w2) * ds(1)  # Inlet
              + gamma_c * inner(w1 + u_par*rotate(n), w2) * ds(4)  # Obstacle
            )
        else:
            h1_sym_ext = (
                gamma_c * inner(w1, w2) * ds(1)  # Inlet
              + gamma_c * inner(w1, w2) * ds(4)  # Obstacle
            )
        return h1_sym_bulk + h1_sym_int + h1_sym_ext



    # -----------------
    #    Data output
    # -----------------
    sigma = 2/Re_c * dev(u) + (dot(u,u)/2 - p) * Identity(2)
    def print_data(u_par=0):
        energy = 1/2 * assemble(inner(u, u) * dx)
        enstrophy = assemble(h1_sym(u, u, u_par=u_par))
        drag = - assemble(dot(as_vector([1,0]), dot(sigma, n)) * ds(4))
        lift = - assemble(dot(as_vector([0,1]), dot(sigma, n)) * ds(4))
        print(RED % f"Energy: {energy:.4e} | Enstrophy: {enstrophy:.4e} | Drag: {drag:.4e} | Lift: {lift:.4e}")


    # --------------------------
    #    Initial Stokes solve
    # --------------------------
    # Residual objects
    vorticity = 1 / Re_c * inner(grad(w), grad(c)) * dx
    diss_aux  = 2 / Re_c * h1_sym(u, rotate(grad(c)))
    pressure  = inner(p, div(v)) * dx
    diss      = 2 / Re_c * h1_sym(u, v)
    incomp    = inner(div(u), q) * dx

    # Residual
    F = (
        vorticity + diss_aux
      - pressure + diss
      - incomp
    )

    # Boundary conditions
    def partial_slip_omega_bc(index, radius=None):
        if radius is None:
            return EquationBC(inner(w + gamma_c*Re_c * dot(u, rotate(n)), c) * ds(index) == 0, z, index, V=Z.sub(0))
        else:
            return EquationBC(inner(w + gamma_c*Re_c * dot(u, rotate(n)) + 2/radius * dot(u, rotate(n)), c) * ds(index) == 0, z, index, V=Z.sub(0))
    bcs_base = [
        partial_slip_omega_bc(1),  # Inlet
        DirichletBC(Z.sub(0), 0, 2),  # Outlet
        DirichletBC(Z.sub(0), 0, 3),  # Walls
        DirichletBC(Z.sub(1), as_vector([1.0, 0.0]), 1),  # Inlet
        DirichletBC(Z.sub(1), as_vector([1.0, 0.0]), 2),  # Outlet
        DirichletBC(Z.sub(1), as_vector([0.0, 0.0]), 3),  # Walls
        DirichletBC(Z.sub(1), as_vector([0.0, 0.0]), 4),  # Obstacle
    ]
    bcs = bcs_base + [partial_slip_omega_bc(4, radius=0.5)]

    # PVD setup
    if save:
        !rm -rf velocity*
        outfile = VTKFile("velocity.pvd")

    # Solve
    print(GREEN % f"Setting up ICs for t = 0:")
    solve(F == 0, z, bcs=bcs, solver_parameters=sp)
    print_data()
    if save: outfile.write(w_sol, u_sol, p_sol)
    u_old.assign(u_sol)



    # ------------------------------
    #    Full Navier-Stokes solve
    # ------------------------------
    # Functions
    u_mid = theta*u + (1-theta)*u_old

    # Residual objects
    # adv_aux_ext = 1/2 * inner(dot(u, u), dot(rotate(grad(c)), n)) * ds(2)  # Outlet
    def diss_aux(u_par): return 2 / Re_c * h1_sym(u_mid, rotate(grad(c)), u_par=u_par)
    transient = 1 / dt_c * inner((u - u_old), v) * dx
    adv       = inner(w * rotate(u_mid), v) * dx
    # adv_ext = 1/2 * inner(dot(u, u), dot(v, n)) * ds(2)  # Outlet
    def diss(u_par): return 2 / Re_c * h1_sym(u_mid, v, u_par=u_par)

    # Boundary conditions
    def partial_slip_omega_bc(index, radius=None, u_par=0):
        if radius is None:
            return EquationBC(inner(w + gamma_c*Re_c * (dot(u_mid, rotate(n)) - u_par), c) * ds(index) == 0, z, index, V=Z.sub(0))
        else:
            return EquationBC(inner(w + gamma_c*Re_c * (dot(u_mid, rotate(n)) - u_par) + 2/radius * dot(u_mid, rotate(n)), c) * ds(index) == 0, z, index, V=Z.sub(0))
    def bcs(u_par=0): return bcs_base + [partial_slip_omega_bc(4, radius=0.5, u_par=u_par)]

    # Residual
    def F(u_par=0):
        return (
            vorticity + diss_aux(u_par)
          + transient + adv - pressure + diss(u_par)
          - incomp
        )

    # Time loop
    t = 0.0
    step = 0
    while step < round(T/dt):
        t += dt
        step += 1
        print(GREEN % f"Solving for t = {t:.4e}:")
        if (osc_amp is None) or (t >= 1/0.212):
            solve(F() == 0, z, bcs=bcs(), solver_parameters=sp)
            print_data()
        else:
            u_par = osc_amp_c*sin(0.212*2*pi*(t - (1-theta)*dt))
            solve(F(u_par=u_par) == 0, z, bcs=bcs(u_par=u_par), solver_parameters=sp)
            print_data(u_par=u_par)
        if save: outfile.write(w_sol, u_sol, p_sol)
        u_old.assign(u_sol)

In [7]:
def download():
    import zipfile
    import glob
    import os
    from google.colab import files

    # Zip the .pvd and all .vtu files
    zip_name = "velocity_data.zip"
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        if os.path.exists("velocity.pvd"):
            zipf.write("velocity.pvd")

        # The .vtu files are stored in a folder called 'velocity'
        if os.path.isdir("velocity"):
            for root, _, files_list in os.walk("velocity"):
                for file in files_list:
                    if file.endswith(".vtu"):
                        file_path = os.path.join(root, file)
                        zipf.write(file_path, arcname=file_path)

    files.download(zip_name)
    print(BLUE % f"Download complete!")

In [10]:
navier_stokes(Re=10000.0, sigma=50.0, gamma=0.02, dt=0.1, T=5.0, theta=0.5, osc_amp=0.0, save=True)
download()

Setting up ICs for t = 0:
  0 SNES Function norm 2.017293087345e+01
  1 SNES Function norm 3.061164415153e-04
  2 SNES Function norm 1.805920758781e-13
  Nonlinear firedrake_12_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
Drag: 5.3958e-03 | Lift: 1.5776e-06
Solving for t = 1.0000e-01:
  0 SNES Function norm 1.215970845213e+01
  1 SNES Function norm 2.077236816089e-01
  2 SNES Function norm 2.679991737977e-04
  3 SNES Function norm 4.178542283759e-08
  Nonlinear firedrake_13_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 3
Drag: 4.6988e-01 | Lift: -8.2736e-05
Solving for t = 2.0000e-01:
  0 SNES Function norm 6.928337439252e+00
  1 SNES Function norm 3.336044905534e-01
  2 SNES Function norm 2.488003972077e-03
  3 SNES Function norm 9.682141976643e-06
  4 SNES Function norm 2.899392968747e-10
  Nonlinear firedrake_14_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 4
Drag: 1.0408e+00 | Lift: -1.8435e-03
Solving for t = 3.0000e-01:
  0 SNES Functi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download complete!
